| Chapter  | Colab   | Kaggle          | Gradient      | Studio Lab             | Binder             |
|:---------|:--------|:----------------|:--------------|:-----------------------|:-------------------|
| [Chapter 1: Pytorch DP Demo](1_privacy/Chapter_1_Pytorch_DP_Demo.ipynb)                         | [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/Chapter_1_Pytorch_DP_Demo.ipynb)            | [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/Chapter_1_Pytorch_DP_Demo.ipynb)            | [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/Chapter_1_Pytorch_DP_Demo.ipynb)            | [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/Chapter_1_Pytorch_DP_Demo.ipynb)            | [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/matthew-mcateer/practicing_trustworthy_machine_learning/HEAD?urlpath=https%3A%2F%2Fgithub.com%2Fmatthew-mcateer%2Fpracticing_trustworthy_machine_learning%2Fblob%2Fmain%2F1_privacy%2FChapter_1_Pytorch_DP_Demo.ipynb)              |

<!--
Originally found on GitHub at https://github.com/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/Chapter_1_Pytorch_DP_Demo.ipynb
-->

